# Mejorar Recall - Detección de Fraude

**Autor**: Ing. Daniel Varela Perez  
**Email**: bedaniele0@gmail.com  
**Teléfono**: +52 55 4189 3428  
**Fecha**: 24 de Septiembre, 2025

**Objetivo**: Mejorar el recall del modelo manteniendo alta precision

In [1]:
# Setup básico
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, 
    confusion_matrix,
    precision_recall_curve,
    roc_curve,
    f1_score,
    precision_score,
    recall_score
)
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports completados")

✅ Imports completados


In [2]:
# Cargar modelo y datos
print("📂 Cargando modelo y datos...")

# Cargar modelo anterior
simple_model = joblib.load('../models/simple_fraud_model.pkl')
print("✅ Modelo simple cargado")

# Cargar datos
train_df = pd.read_parquet('../data/processed/train_clean.parquet')
test_df = pd.read_parquet('../data/processed/test_clean.parquet')

# Preparar datos
feature_cols = [col for col in train_df.columns if col not in ['Class', 'Time']]
target_col = 'Class'

X_train = train_df[feature_cols]
y_train = train_df[target_col]
X_test = test_df[feature_cols]
y_test = test_df[target_col]

print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Fraudes train: {y_train.sum()} ({y_train.sum()/len(y_train)*100:.4f}%)")
print(f"Fraudes test: {y_test.sum()} ({y_test.sum()/len(y_test)*100:.4f}%)")
print("✅ Datos preparados")

📂 Cargando modelo y datos...
✅ Modelo simple cargado
Train: (172090, 39), Test: (53130, 39)
Fraudes train: 360.0 (0.2092%)
Fraudes test: 71.0 (0.1336%)
✅ Datos preparados


In [18]:
# Performance del modelo baseline
print("📊 PERFORMANCE BASELINE")
print("=" * 30)

# Predicciones baseline
y_pred_baseline = simple_model.predict(X_test)
y_proba_baseline = simple_model.predict_proba(X_test)[:, 1]

# Métricas baseline
precision_baseline = precision_score(y_test, y_pred_baseline)
recall_baseline = recall_score(y_test, y_pred_baseline)
f1_baseline = f1_score(y_test, y_pred_baseline)

print(f"Precision: {precision_baseline:.4f}")
print(f"Recall: {recall_baseline:.4f}")
print(f"F1-Score: {f1_baseline:.4f}")
print(f"Fraudes detectados: {y_pred_baseline.sum()}/{y_test.sum()}")

# Matriz de confusión baseline
cm_baseline = confusion_matrix(y_test, y_pred_baseline)
print(f"\nMatriz Confusión Baseline:")
print(cm_baseline)
print("✅ Baseline evaluado")

📊 PERFORMANCE BASELINE
Precision: 1.0000
Recall: 0.1268
F1-Score: 0.2250
Fraudes detectados: 9.0/71.0

Matriz Confusión Baseline:
[[53059     0]
 [   62     9]]
✅ Baseline evaluado


In [19]:
# Ajustar threshold para mejorar recall
print("🎯 AJUSTANDO THRESHOLD PARA MEJOR RECALL")
print("=" * 45)

# Probar diferentes thresholds
thresholds = [0.5, 0.3, 0.2, 0.1, 0.05, 0.03, 0.01]
results = []

for threshold in thresholds:
    # Predicciones con threshold personalizado
    y_pred_thresh = (y_proba_baseline >= threshold).astype(int)
    
    # Métricas
    precision = precision_score(y_test, y_pred_thresh, zero_division=0)
    recall = recall_score(y_test, y_pred_thresh, zero_division=0)
    f1 = f1_score(y_test, y_pred_thresh, zero_division=0)
    
    # Fraudes detectados
    detected = y_pred_thresh.sum()
    true_frauds = y_test.sum()
    
    results.append({
        'threshold': threshold,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'detected': detected,
        'true_frauds': int(true_frauds)
    })
    
    print(f"Threshold {threshold:4.2f}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}, Detectados={detected}/{int(true_frauds)}")

# Convertir a DataFrame para análisis
results_df = pd.DataFrame(results)
print("\n✅ Análisis de thresholds completado")

🎯 AJUSTANDO THRESHOLD PARA MEJOR RECALL
Threshold 0.50: Precision=1.000, Recall=0.155, F1=0.268, Detectados=11/71
Threshold 0.30: Precision=1.000, Recall=0.662, F1=0.797, Detectados=47/71
Threshold 0.20: Precision=0.944, Recall=0.718, F1=0.816, Detectados=54/71
Threshold 0.10: Precision=0.885, Recall=0.761, F1=0.818, Detectados=61/71
Threshold 0.05: Precision=0.767, Recall=0.789, F1=0.778, Detectados=73/71
Threshold 0.03: Precision=0.479, Recall=0.817, F1=0.604, Detectados=121/71
Threshold 0.01: Precision=0.048, Recall=0.873, F1=0.091, Detectados=1299/71

✅ Análisis de thresholds completado


In [20]:
# Seleccionar mejor threshold balanceando precision y recall
print("🏆 SELECCIONAR MEJOR THRESHOLD")
print("=" * 35)

# Criterios para selección:
# 1. Recall > 50% (detectar al menos la mitad de fraudes)
# 2. Precision > 10% (evitar demasiadas falsas alarmas)
# 3. Maximizar F1-Score

# Filtrar candidatos viables
viable = results_df[
    (results_df['recall'] >= 0.50) & 
    (results_df['precision'] >= 0.10)
]

if len(viable) > 0:
    # Seleccionar el de mejor F1-Score
    best_idx = viable['f1'].idxmax()
    best_threshold = results_df.loc[best_idx, 'threshold']
    best_result = results_df.loc[best_idx]
else:
    # Si no hay viables, seleccionar el de mejor recall con precision >= 5%
    viable_relaxed = results_df[results_df['precision'] >= 0.05]
    if len(viable_relaxed) > 0:
        best_idx = viable_relaxed['recall'].idxmax()
        best_threshold = results_df.loc[best_idx, 'threshold']
        best_result = results_df.loc[best_idx]
    else:
        # Fallback: mejor F1 general
        best_idx = results_df['f1'].idxmax()
        best_threshold = results_df.loc[best_idx, 'threshold']
        best_result = results_df.loc[best_idx]

print(f"🎯 MEJOR THRESHOLD: {best_threshold}")
print(f"📊 MÉTRICAS:")
print(f"  • Precision: {best_result['precision']:.4f}")
print(f"  • Recall: {best_result['recall']:.4f}")
print(f"  • F1-Score: {best_result['f1']:.4f}")
print(f"  • Fraudes detectados: {best_result['detected']}/{best_result['true_frauds']}")

# Aplicar mejor threshold
y_pred_best = (y_proba_baseline >= best_threshold).astype(int)
print("\n✅ Mejor threshold seleccionado")

🏆 SELECCIONAR MEJOR THRESHOLD
🎯 MEJOR THRESHOLD: 0.1
📊 MÉTRICAS:
  • Precision: 0.8852
  • Recall: 0.7606
  • F1-Score: 0.8182
  • Fraudes detectados: 61.0/71.0

✅ Mejor threshold seleccionado


In [21]:
# Entrenar modelo con SMOTE para mejor recall
print("⚖️ ENTRENANDO MODELO CON SMOTE")
print("=" * 35)

# Aplicar SMOTE
print("🔄 Aplicando SMOTE...")
smote = SMOTE(random_state=42, k_neighbors=min(5, int(y_train.sum())-1))
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f"Original: {len(y_train):,} muestras ({y_train.sum()} fraudes)")
print(f"SMOTE: {len(y_train_smote):,} muestras ({y_train_smote.sum()} fraudes)")
print(f"Ratio después SMOTE: {(len(y_train_smote) - y_train_smote.sum()) / y_train_smote.sum():.1f}:1")

# Entrenar modelo con datos balanceados
print("\n🚀 Entrenando Random Forest con SMOTE...")
rf_smote = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight=None,  # No necesario con SMOTE
    n_jobs=-1
)

rf_smote.fit(X_train_smote, y_train_smote)
print("✅ Modelo SMOTE entrenado")

⚖️ ENTRENANDO MODELO CON SMOTE
🔄 Aplicando SMOTE...
Original: 172,090 muestras (360.0 fraudes)
SMOTE: 343,460 muestras (171730.0 fraudes)
Ratio después SMOTE: 1.0:1

🚀 Entrenando Random Forest con SMOTE...
✅ Modelo SMOTE entrenado


In [22]:
# Evaluar modelo SMOTE
print("📊 EVALUACIÓN MODELO SMOTE")
print("=" * 30)

# Predicciones modelo SMOTE
y_pred_smote = rf_smote.predict(X_test)
y_proba_smote = rf_smote.predict_proba(X_test)[:, 1]

# Métricas modelo SMOTE
precision_smote = precision_score(y_test, y_pred_smote)
recall_smote = recall_score(y_test, y_pred_smote)
f1_smote = f1_score(y_test, y_pred_smote)

print(f"SMOTE Precision: {precision_smote:.4f}")
print(f"SMOTE Recall: {recall_smote:.4f}")
print(f"SMOTE F1-Score: {f1_smote:.4f}")
print(f"Fraudes detectados: {y_pred_smote.sum()}/{y_test.sum()}")

# Matriz de confusión SMOTE
cm_smote = confusion_matrix(y_test, y_pred_smote)
print(f"\nMatriz Confusión SMOTE:")
print(cm_smote)
print("\n✅ Modelo SMOTE evaluado")

📊 EVALUACIÓN MODELO SMOTE
SMOTE Precision: 1.0000
SMOTE Recall: 0.3521
SMOTE F1-Score: 0.5208
Fraudes detectados: 25.0/71.0

Matriz Confusión SMOTE:
[[53059     0]
 [   46    25]]

✅ Modelo SMOTE evaluado


In [23]:
# Comparación de enfoques
print("🔍 COMPARACIÓN DE ENFOQUES")
print("=" * 40)

comparison = pd.DataFrame({
    'Enfoque': [
        'Baseline (threshold=0.5)',
        f'Threshold Ajustado ({best_threshold})',
        'SMOTE + Random Forest'
    ],
    'Precision': [
        precision_baseline,
        best_result['precision'],
        precision_smote
    ],
    'Recall': [
        recall_baseline,
        best_result['recall'],
        recall_smote
    ],
    'F1_Score': [  # CORREGIDO: Sin guión
        f1_baseline,
        best_result['f1'],
        f1_smote
    ],
    'Fraudes_Detectados': [
        y_pred_baseline.sum(),
        best_result['detected'],
        y_pred_smote.sum()
    ]
})

# DEBUG: Ver columnas exactas del DataFrame
print("DEBUG - Columnas disponibles en comparison:")
print(comparison.columns.tolist())
print("\nPrimeras filas del DataFrame:")
print(comparison.head())

print("\nTabla de comparación:")
print(comparison.round(4))

# Determinar mejor enfoque - CORREGIDO
best_approach_idx = comparison['F1_Score'].idxmax()  # Sin guión
best_approach = comparison.iloc[best_approach_idx]['Enfoque']

print(f"\n🏆 MEJOR ENFOQUE: {best_approach}")
print(f"📈 F1-Score: {comparison.iloc[best_approach_idx]['F1_Score']:.4f}")
print(f"🎯 Recall: {comparison.iloc[best_approach_idx]['Recall']:.4f}")
print(f"⚖️ Precision: {comparison.iloc[best_approach_idx]['Precision']:.4f}")
print("\n✅ Comparación completada")

🔍 COMPARACIÓN DE ENFOQUES
DEBUG - Columnas disponibles en comparison:
['Enfoque', 'Precision', 'Recall', 'F1_Score', 'Fraudes_Detectados']

Primeras filas del DataFrame:
                    Enfoque  Precision    Recall  F1_Score  Fraudes_Detectados
0  Baseline (threshold=0.5)   1.000000  0.126761  0.225000                 9.0
1  Threshold Ajustado (0.1)   0.885246  0.760563  0.818182                61.0
2     SMOTE + Random Forest   1.000000  0.352113  0.520833                25.0

Tabla de comparación:
                    Enfoque  Precision  Recall  F1_Score  Fraudes_Detectados
0  Baseline (threshold=0.5)     1.0000  0.1268    0.2250                 9.0
1  Threshold Ajustado (0.1)     0.8852  0.7606    0.8182                61.0
2     SMOTE + Random Forest     1.0000  0.3521    0.5208                25.0

🏆 MEJOR ENFOQUE: Threshold Ajustado (0.1)
📈 F1-Score: 0.8182
🎯 Recall: 0.7606
⚖️ Precision: 0.8852

✅ Comparación completada


In [ ]:
# Guardar mejor modelo y configuración
print("💾 GUARDANDO MEJOR CONFIGURACIÓN")
print("=" * 37)

# Determinar cual guardar basado en F1-Score
if f1_smote >= best_result['f1']:
    # SMOTE es mejor
    best_model = rf_smote
    best_method = "SMOTE"
    best_f1_final = f1_smote
    best_recall_final = recall_smote
    best_precision_final = precision_smote
    model_path = '../models/improved_recall_smote_model.pkl'
else:
    # Threshold ajustado es mejor
    best_model = simple_model  # Usar el modelo original con threshold
    best_method = f"Threshold {best_threshold}"
    best_f1_final = best_result['f1']
    best_recall_final = best_result['recall']
    best_precision_final = best_result['precision']
    model_path = '../models/improved_recall_threshold_model.pkl'
    
    # Guardar también el threshold
    import json
    config = {'best_threshold': float(best_threshold)}
    with open('../models/threshold_config.json', 'w') as f:
        json.dump(config, f)
    print(f"✅ Threshold guardado: {best_threshold}")

# Crear directorio models si no existe
import os
os.makedirs('../models', exist_ok=True)

# Guardar modelo
joblib.dump(best_model, model_path)
print(f"✅ Modelo guardado: {model_path}")

# Calcular mejora porcentual
recall_improvement = (best_recall_final / recall_baseline - 1) * 100 if recall_baseline > 0 else 0

# Resumen final
print(f"\n🎯 RESUMEN FINAL - MEJORA DE RECALL:")
print(f"• Método ganador: {best_method}")
print(f"• F1-Score: {best_f1_final:.4f} (vs {f1_baseline:.4f} baseline)")
print(f"• Recall: {best_recall_final:.4f} (vs {recall_baseline:.4f} baseline)")
print(f"• Precision: {best_precision_final:.4f} (vs {precision_baseline:.4f} baseline)")
print(f"• Mejora en recall: {recall_improvement:.1f}%")
print(f"• Modelo guardado exitosamente")

print(f"\n👨‍💻 Desarrollado por: Ing. Daniel Varela Perez")
print(f"📧 bedaniele0@gmail.com | 📱 +52 55 4189 3428")
print("\n✅ PROCESO COMPLETADO")